In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
!python -m pip show scikit-learn 

Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: joblib, scipy, numpy
Required-by: yellowbrick, sklearn, sklearn-pandas, mlxtend, lightgbm, librosa, imbalanced-learn


Import data

In [2]:
%cd /content/drive/MyDrive/nlp-getting-started/
!ls

/content/drive/MyDrive/nlp-getting-started
sample_submission.csv  test.csv  train.csv


### Explore data

In [29]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(train_df.head)
print(train_df.iloc[0])
print(train_df[train_df["target"] == 0]["text"].values[0])
print(train_df[train_df["target"] == 1]["text"].values[0])

print(type(train_df))
print(train_df.size)
print(test_df.size)

<bound method NDFrame.head of          id keyword  ...                                               text target
0         1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1         4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2         5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3         6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4         7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1
...     ...     ...  ...                                                ...    ...
7608  10869     NaN  ...  Two giant cranes holding a bridge collapse int...      1
7609  10870     NaN  ...  @aria_ahrary @TheTawniest The out of control w...      1
7610  10871     NaN  ...  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...      1
7611  10872     NaN  ...  Police investigating after an e-bike collided ...      1
7612  10873     NaN  ...  The Latest: More Homes Razed by

### Feature Extraction


In [4]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [5]:
count_vectorizer = feature_extraction.text.CountVectorizer()
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [6]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [7]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

In [19]:
features = count_vectorizer.get_feature_names()
print(len(features))

21637


In [26]:
test_vectors = count_vectorizer.transform(test_df["text"])
print(type(test_vectors))
print(test_vectors.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(3263, 21637)


### Linear Model

In [31]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
classifier = linear_model.RidgeClassifier()

In [34]:
## Cross-validation
scores = model_selection.cross_val_score(classifier, train_vectors, train_df["target"], cv=3, scoring="f1")
print(scores)

[0.59453669 0.56498283 0.64082434]


In [35]:
classifier.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [36]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = classifier.predict(test_vectors)
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [38]:
sample_submission.to_csv("submission_vishwa.csv", index=False)